In [5]:
#!pip install tensorflow

In [35]:
import os
import math
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory

from PIL import Image, ImageOps

from IPython.display import display

In [36]:
def downsample(img, r=0.1):
    """ downsample an image by a percentage
        :param img, a PIL Image format
        :r ratio, the percentage new image size
    """
    width, height = img.size
    w = int(width*r)
    h = int(height*r)
    if 'P' in img.mode: # check if image is a palette type
        img = img.convert("RGB") # convert it to RGB
        img = img.resize((w,h),Image.ANTIALIAS) # resize it
        img = img.convert("P",dither=Image.NONE, palette=Image.ADAPTIVE) 
           #convert back to palette
    else:
        img = img.resize((w,h),Image.ANTIALIAS) # regular resize
    return img


In [45]:
images = []
for root, dirs, files in os.walk("data/Holopix50k/train/left"):
    path = root.split(os.sep)
    #print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        images.append(root+"/"+file)
        #print(len(path) * '---', file)

In [44]:
len(images)

41746

In [46]:
images[0]

'data/Holopix50k/train/left/-LVjXbOW-nEA4Ii4FYtQ_left.jpg'

In [57]:
im = Image.open(images[1])
im = im.crop((0, 0, im.size[1], im.size[1]))


In [58]:
im.size

(720, 720)

In [59]:
im_small = downsample(im)
im_small.size

(72, 72)

In [69]:
def define_model(input_size=(72,72,3), upscale_factor=6):
    conv_args = {
        "activation": "relu",
        "kernel_initializer": "Orthogonal",
        "padding": "same",
    }
    inputs = keras.Input(shape=input_size)
    x = layers.Conv2D(64, 5, **conv_args)(inputs)
    x = layers.Conv2D(64, 3, **conv_args)(x)
    x = layers.Conv2D(32, 3, **conv_args)(x)
    x = layers.Conv2D(1 * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

    return keras.Model(inputs, outputs)

In [71]:
def get_model(upscale_factor=3, channels=1):
    conv_args = {
        "activation": "relu",
        "kernel_initializer": "Orthogonal",
        "padding": "same",
    }
    inputs = keras.Input(shape=(72,72, channels))
    x = layers.Conv2D(64, 5, **conv_args)(inputs)
    x = layers.Conv2D(64, 3, **conv_args)(x)
    x = layers.Conv2D(32, 3, **conv_args)(x)
    x = layers.Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

    return keras.Model(inputs, outputs)


In [72]:
model = get_model()

ValueError: Dimension size must be evenly divisible by 9 but is 1 for '{{node tf.nn.depth_to_space_4/DepthToSpace}} = DepthToSpace[T=DT_FLOAT, block_size=3, data_format="NHWC"](Placeholder)' with input shapes: [?,9,72,1].